<a href="https://colab.research.google.com/github/souritmaji/Alcovex/blob/main/Future_TextPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

import heapq

import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer

from keras.models import Sequential, load_model

from keras.layers.core import Dense, Activation

from keras.layers import LSTM

import pickle

from tensorflow.keras.optimizers import RMSprop

In [3]:
path = 'new.txt'
text = open(path).read().lower()
print('length of the corpus is: :', len(text))


length of the corpus is: : 119163


In [4]:
import string

In [5]:
file = open("new.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)


In [6]:
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
words = tokenizer.tokenize(text)

In [7]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [8]:
unique_words

array(['!', '!"', '!",', ..., 'young', 'your', 'yourself'], dtype='<U17')

In [10]:
LENGTH_WORD = 5
next_words = []
prev_words = []
for j in range(len(words) - LENGTH_WORD):
     prev_words.append(words[j:j + LENGTH_WORD])
     next_words.append(words[j + LENGTH_WORD])
     
print(prev_words[15])
print(next_words[15])

['in', 'his', 'bed', 'into', 'a']
horrible


In [11]:
X = np.zeros((len(prev_words), LENGTH_WORD, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [12]:
count = 0
for i, each_words in enumerate(prev_words):
  #  print("First : " ,each_words)
   for j, each_word in enumerate(each_words):
        # if count == 10:
        #   break
        # print("Second : ", each_word)
        count += 1
        X[i, j, unique_word_index[each_word]] = 1
   Y[i, unique_word_index[next_words[i]]] = 1
  #  if count == 10:
  #         break




In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH_WORD, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.1, batch_size=64, epochs=100, shuffle=True).history

Epoch 1/100
348/348 [==============================] - 29s 73ms/step - loss: 2.1002 - accuracy: 0.7771 - val_loss: 7.3653 - val_accuracy: 0.0782
Epoch 2/100
348/348 [==============================] - 25s 72ms/step - loss: 1.9652 - accuracy: 0.7913 - val_loss: 7.3729 - val_accuracy: 0.0749
Epoch 3/100
348/348 [==============================] - 25s 71ms/step - loss: 1.9474 - accuracy: 0.7934 - val_loss: 7.3899 - val_accuracy: 0.0741
Epoch 4/100
348/348 [==============================] - 25s 72ms/step - loss: 1.9317 - accuracy: 0.7959 - val_loss: 7.3825 - val_accuracy: 0.0790
Epoch 5/100
348/348 [==============================] - 25s 71ms/step - loss: 1.9188 - accuracy: 0.7957 - val_loss: 7.4125 - val_accuracy: 0.0810
Epoch 6/100
348/348 [==============================] - 25s 72ms/step - loss: 1.9213 - accuracy: 0.7945 - val_loss: 7.4189 - val_accuracy: 0.0737
Epoch 7/100
348/348 [==============================] - 25s 71ms/step - loss: 1.9159 - accuracy: 0.7947 - val_loss: 7.3551 - val_ac

In [ ]:
model.save('next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('next_word_model.h5')
history = pickle.load(open("history.p", "rb"))